In [1]:
import numpy as np
import pandas as pd
import sys
import pickle
import xml.etree.ElementTree as ET
sys.path.append('../../scripts/beastAnalysis')  # need to import from working directory
import beastmatrix as bm

In [10]:
#################### Parse and reformat input #####################
main = {}
main['burnin'] = 2500000
main['logfile'] = '../../beast/main/full_dataset/discreteTraits/discreteTraits_2016-10-25.log'
main['xmlfile'] = '../../beast/main/full_dataset/discreteTraits/discreteTraits_2016-10-25.xml'
main['trait'] = 'host'
main['prior'] = 20

resampled = {}
resampled['burnin'] = 20000000
resampled['logfile'] = '../../beast/supplement/discreteTraits/discreteTraits_2016-12-27.log'
resampled['xmlfile'] = '../../beast/supplement/discreteTraits/discreteTraits_2016-12-27.xml'
resampled['trait'] = 'host'
resampled['prior'] = 20

pruned = {}
pruned['burnin'] = 20000
pruned['logfile'] = '../../beast/main/pruned_segments/discreteTraits/pruned_combined_10mBurnin.log'
pruned['xmlfile'] = '../../beast/main/pruned_segments/discreteTraits/discreteTraits_2017-04-03.xml'
pruned['trait'] = 'host'
pruned['prior'] = 23


datasets = [main, resampled, pruned]

common_names = pickle.load(open('../../data/hosts/common_names.p', 'rb'))

In [11]:
resampled['log_data'] = pd.read_csv(resampled['logfile'], skiprows=2, sep="\t", index_col='state')	#parse the log file as a df
main['log_data'] = pd.read_csv(main['logfile'], skiprows=2, sep="\t", index_col='state')
pruned['log_data'] = pd.read_csv(pruned['logfile'], comment='#', sep="\t", index_col='state')

main_xml = ET.parse(main['xmlfile'])
main_root = main_xml.getroot()												#get the list of states, in order, from the xml
main_states = [ i for i in main_root.findall('generalDataType') if i.get('id').startswith(main['trait'])][0]
main['state_list'] = [ common_names[state.get('code') ] for state in main_states]
main['n_demes'] = len(main['state_list'])

resampled_xml = ET.parse(resampled['xmlfile'])
resampled_root = resampled_xml.getroot()												#get the list of states, in order, from the xml
resampled_states = [ i for i in resampled_root.findall('generalDataType') if i.get('id').startswith(resampled['trait'])][0]
resampled['state_list'] = [ common_names[state.get('code') ] for state in resampled_states]
resampled['n_demes'] = len(resampled['state_list'])

pruned_xml = ET.parse(pruned['xmlfile'])
pruned_root = pruned_xml.getroot()												#get the list of states, in order, from the xml
pruned_states = [ i for i in pruned_root.findall('generalDataType') if i.get('id').startswith(pruned['trait'])][0]
pruned['state_list'] = [ common_names[state.get('code') ] for state in pruned_states]
pruned['n_demes'] = len(pruned['state_list'])


################# Deal with burnin, make data structures ###############################

for d in datasets:
    assert d['burnin'] in d['log_data'].index, (d['burnin'], d['log_data'].index)
    burnin_indices = range(0, d['log_data'].index.get_loc(d['burnin'])+1)
    d['log_data'].drop(d['log_data'].index[burnin_indices], inplace=True)
    d['log_data'] = d['log_data'].append(pd.DataFrame(d['log_data'].mean(), columns=['avg']).T)
    d['bf'] = pd.DataFrame(dtype = 'float', index=d['state_list'], columns=d['state_list'])
    d['ar'] = pd.DataFrame(dtype = 'float', index=d['state_list'], columns=d['state_list'])
    d['indicator'] = pd.DataFrame(dtype = 'float', index=d['state_list'], columns=d['state_list'])

    indicator_series = []
    actualRates_series = []
    bf_series = []
    for column, series in d['log_data'].iteritems():					#pull the average posterior values from appropriate columns in order
        if 'actualRates' in column and main['trait'] in column:			#we will then use this to fill our matrices.
            actualRates_series.append(series['avg'])
        elif 'indicator' in column and main['trait'] in column:
            bf_series.append(bm.find_bf(series['avg'], d['prior'], d['n_demes']))	#convert indicators to bayes factors while we're at it
            indicator_series.append(series['avg'])
        else:
            continue

    d['bf'] = bm.fill_rates_matrix(d['bf'], pd.Series(bf_series))
    d['ar'] = bm.fill_rates_matrix(d['ar'], pd.Series(actualRates_series))
    d['indicator'] = bm.fill_rates_matrix(d['indicator'], pd.Series(indicator_series))


In [12]:
data = []
sha#DF4327_states = set(main['state_list']).intersection(set(resampled['state_list']))
for from_state in sha#DF4327_states:
    for to_state in sha#DF4327_states:
        if from_state != to_state:
            data.append({
            'from_%s'%main['trait']: from_state,
            'to_%s'%main['trait']: to_state,
            'main_bf': main['bf'].at[from_state, to_state],
            'main_ar': main['ar'].at[from_state, to_state],
            'main_indicator': main['indicator'].at[from_state, to_state],
            'resampled_bf': resampled['bf'].at[from_state, to_state],
            'resampled_ar': resampled['ar'].at[from_state, to_state],
            'resampled_indicator': resampled['indicator'].at[from_state, to_state],
            'pruned_bf': pruned['bf'].at[from_state, to_state],
            'pruned_ar': pruned['ar'].at[from_state, to_state],
            'pruned_indicator': pruned['indicator'].at[from_state, to_state],
                    
            })

compare = pd.DataFrame(data, columns = ['from_%s'%main['trait'], 'to_%s'%main['trait'], 
                                        'main_indicator', 'main_bf', 'main_ar',
                                       'resampled_indicator', 'resampled_bf', 'resampled_ar', 
                                        'pruned_indicator', 'pruned_bf', 'pruned_ar'])

# compare['main_bf'] = compare['main_bf'].map(np.log)
# compare['resampled_bf'] = compare['resampled_bf'].map(np.log)

In [5]:
compare.sort_values('resampled_bf', inplace=True)
compare.to_csv('../../beast/compare_discreteTraits.csv')
compare

from_host              to_host  main_indicator       main_bf  \
150  Red-capped_Mangabey                Human        0.014889  4.020275e-01   
205            Chimp_pts               Vervet        0.029556  8.101185e-01   
78             Chimp_ptt               Vervet        0.031778  8.730291e-01   
81             Chimp_ptt       Sabaeus_Monkey        0.033111  9.109142e-01   
140  Red-capped_Mangabey               Baboon        0.038889  1.076298e+00   
208            Chimp_pts       Sabaeus_Monkey        0.031111  8.541256e-01   
35              Mandrill       Sooty_Mangabey        0.025111  6.851579e-01   
77             Chimp_ptt       Sooty_Mangabey        0.025556  6.976027e-01   
38              Mandrill                Human        0.014444  3.898507e-01   
36              Mandrill               Vervet        0.026000  7.100588e-01   
74             Chimp_ptt     DeBrazzas_Monkey        0.031778  8.730291e-01   
82             Chimp_ptt      Tantalus_Monkey        0.027556  7.537449e-01   
207            Chimp_pts                Human        0.024000  6.540956e-01   
204            Chimp_pts       Sooty_Mangabey        0.021333  5.798337e-01   
165                Human       Sabaeus_Monkey        0.048889  1.367287e+00   
162                Human       Sooty_Mangabey        0.042667  1.185512e+00   
91               Gorilla       Sooty_Mangabey        0.043556  1.211335e+00   
52        Colobus_Monkey                Human        0.034889  9.615905e-01   
209            Chimp_pts      Tantalus_Monkey        0.029778  8.163966e-01   
120       Sooty_Mangabey               Vervet        0.045333  1.263126e+00   
163                Human               Vervet        0.035778  9.869988e-01   
149  Red-capped_Mangabey               Vervet        0.018000  4.875736e-01   
49        Colobus_Monkey       Sooty_Mangabey        0.038667  1.069900e+00   
123       Sooty_Mangabey       Sabaeus_Monkey        0.038667  1.069900e+00   
166                Human      Tantalus_Monkey        0.034000  9.362290e-01   
158                Human     DeBrazzas_Monkey        0.032667  8.982742e-01   
117       Sooty_Mangabey            Chimp_ptt        0.038222  1.057114e+00   
92               Gorilla               Vervet        0.053333  1.498589e+00   
28              Mandrill               Baboon        0.034667  9.552458e-01   
72             Chimp_ptt             Mandrill        0.098000  2.890019e+00   
..                   ...                  ...             ...           ...   
62      DeBrazzas_Monkey     Red-eared_Guenon        0.115333  3.467819e+00   
155                Human  Western_red_colobus        0.279556  1.032165e+01   
194      Tantalus_Monkey       Sabaeus_Monkey        0.724444  6.993222e+01   
107     Red-eared_Guenon  Red-capped_Mangabey        0.153778  4.833820e+00   
202            Chimp_pts              Gorilla        0.116667  3.513204e+00   
51        Colobus_Monkey  Red-capped_Mangabey        0.363111  1.516552e+01   
57      DeBrazzas_Monkey  Western_red_colobus        0.262000  9.443356e+00   
2                 Baboon       Colobus_Monkey        0.394000  1.729438e+01   
20   Western_red_colobus     Red-eared_Guenon        0.238222  8.318315e+00   
183      Tantalus_Monkey  Western_red_colobus        0.154667  4.866873e+00   
9                 Baboon  Red-capped_Mangabey        0.396667  1.748839e+01   
143  Red-capped_Mangabey       Colobus_Monkey        0.780000  9.430904e+01   
16   Western_red_colobus       Colobus_Monkey        0.256222  9.163365e+00   
23   Western_red_colobus  Red-capped_Mangabey        0.314000  1.217550e+01   
197            Chimp_pts  Western_red_colobus        0.538889  3.108673e+01   
177       Sabaeus_Monkey               Vervet        0.095111  2.795871e+00   
191      Tantalus_Monkey               Vervet        0.980000  1.303393e+03   
201            Chimp_pts            Chimp_ptt        0.976667  1.113395e+03   
75             Chimp_ptt              Gorilla        0.999556  5.

In [13]:
def r_squa#DF4327(xlist, ylist):
    assert len(xlist) == len(ylist), 'unequal number of x and y values'
    sum_x_squa#DF4327 = sum([ x**2 for x in xlist ])
    sum_x = sum(xlist)
    
    sum_y_squa#DF4327 = sum([ y**2 for y in ylist ])
    sum_y = sum(ylist)
    
    sum_xy = sum([ x*y for x,y in zip(xlist,ylist) ])
    n = len(xlist)
    
    r = (n*sum_xy - sum_x*sum_y) / np.sqrt((n*sum_x_squa#DF4327 - sum_x**2)*(n*sum_y_squa#DF4327 - sum_y**2))
    r_squa#DF4327 = r**2
    assert 0 <= r <= 1.0
    assert 0 <= r_squa#DF4327 <= 1.0
    
    return r_squa#DF4327

In [14]:
def sensitivity(bf1, bf2, cutoff=3.0):
    recode = lambda x: x >= cutoff
    bf1_bool = bf1.map(recode)
    bf2_bool = bf2.map(recode)
    comparisons = bf1_bool == bf2_bool
    sensitivity = float(sum(comparisons)) / float(len(comparisons))
    return sensitivity

print sensitivity(compare['main_bf'], compare['pruned_bf'], cutoff=10.0)
print sensitivity(compare['main_bf'], compare['resampled_bf'],cutoff=10.0)
print sensitivity(compare['pruned_bf'], compare['resampled_bf'],cutoff=10.0)


0.909523809524
0.952380952381
0.919047619048


In [15]:
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import scipy

In [25]:
def plot_comparison(xdata, ydata, ax, log, xlabel, ylabel, color):
    if log == True:
        ### For actual rates and bayes factors: Plot raw points on log scale
        points = sns.regplot(xdata, ydata,
                                scatter_kws={'color': color, 'alpha': 0.65}, ax=ax, fit_reg=False) 
        # Do regression on logged points
        reg = sns.regplot(xdata.map(np.log10), ydata.map(np.log10), ax=ax, 
                    line_kws={'color': 'lightgray', 'alpha': 0.5}, scatter_kws={'alpha': 0}, ci=None)
        R_squared = r_squared(xdata.map(np.log10), ydata.map(np.log10))

    else:
        ### For indicators: not on a log scale. Plot points and regression simultaneously.
        reg = sns.regplot(xdata, ydata, ax=ax,
                              scatter_kws={'color': color, 'alpha': 0.65},
                              line_kws={'color': 'lightgray', 'alpha': 0.5}, ci=None)
        R_squared = r_squared(xdata, ydata)

    # Add y=x, R^2
        
    lims = [ min(xdata.min(), ydata.min()), 
               max(xdata.max(), ydata.max())]
    
    ax.plot(lims, lims, '--', color='k', alpha=0.5)
    ax.text(lims[0], lims[1]-0.1*lims[1], 'R^2 = %.2f'%R_squared)
    
    # Set axis limits
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    
    #### Labels
    ax.set_xlabel(xlabel, size=14) 
    ax.set_ylabel(ylabel, size=14)

    ### Format ticks
    ax.tick_params(axis='both', which='major', labelsize=10, width=0.5, length=3, top="off", right="off")
    ax.tick_params(axis='both', which='major', labelsize=10, width=0.5, length=3, top="off", right="off")
    ax.spines['left'].set_linewidth(0.5)
    ax.spines['bottom'].set_linewidth(0.5)

In [26]:
fig, (top, middle, bottom) = plt.subplots(figsize=(7.5,7.5),nrows=3,ncols=3, facecolor='w')


# Compare Actual Rates
top[0].set_title('Actual Rates', size=14, fontweight='bold')    
plot_comparison(compare['main_ar'], compare['resampled_ar'], 
                bottom[0], log=True, color='#547BD3',
                xlabel=' ', ylabel='Resampled Dataset')

plot_comparison(compare['main_ar'], compare['pruned_ar'], 
                middle[0], log=True, color='#547BD3',
                xlabel=' ', ylabel='Pruned Dataset')

plot_comparison(compare['resampled_ar'], compare['pruned_ar'], 
                top[0], log=True, color='#547BD3',
                xlabel=' ', ylabel='Pruned Dataset')


# Compare Bayes Factors
top[1].set_title('Bayes Factors', size=14, fontweight='bold')
plot_comparison(compare['main_bf'], compare['resampled_bf'], 
                bottom[1], log=True, color='#83BA70',
                xlabel='Main Dataset', ylabel=' ')

plot_comparison(compare['main_bf'], compare['pruned_bf'], 
                middle[1], log=True, color='#83BA70',
                xlabel='Main Dataset', ylabel=' ')

plot_comparison(compare['resampled_bf'], compare['pruned_bf'], 
                top[1], log=True, color='#83BA70',
                xlabel='Resampled Dataset', ylabel=' ')

# Compare Indicators
top[2].set_title('Indicators', size=14, fontweight='bold')
plot_comparison(compare['main_indicator'], compare['resampled_indicator'], 
                bottom[2], log=True, color='#DF4327',
                xlabel=' ', ylabel=' ')

plot_comparison(compare['main_indicator'], compare['pruned_indicator'], 
                middle[2], log=True, color='#DF4327',
                xlabel=' ', ylabel=' ')

plot_comparison(compare['resampled_indicator'], compare['pruned_indicator'], 
                top[2], log=True, color='#DF4327',
                xlabel=' ', ylabel=' ')

### Make it pretty
sns.set_style(style='white') 
plt.tight_layout()

### Save our work
plt.savefig('../png/FigS9.png', bbox_inches='tight', dpi=600)

### Figure S9: Comparison of Discrete Trait Analysis Results

Each datapoint represents on of the 210 possible transmissions between each pair of the 15 hosts present in the supplemental, main, and main/pruned datasets. The black dashed line shows y=x; the linear regression is shown in gray.